<a href="https://colab.research.google.com/github/i40-Tools/I40KG-Embeddings/blob/master/Similarity_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I40 standards landscape similarity analysis using embeddings

## Overview

In this notebook, we show the similarity analysis between Industry 4.0 Standards. 
To do so, we create embeddings about the Industry 4.0 Standards Knowledge Graph (I40KG) developed by [Grangel-Gonzales et. al.](https://www.researchgate.net/publication/318208930_The_Industry_40_Standards_Landscape_from_a_Semantic_Integration_Perspective)

The embeddings are located here: [I40 Embeddings](https://github.com/i40-Tools/I40KG-Embeddings/tree/master/logs_sto)

## Initial Configurations
First, let's import the required libraries to perform the similarity analysis.

In [1]:
!git clone https://github.com/i40-Tools/I40KG-Embeddings.git

Cloning into 'I40KG-Embeddings'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 322 (delta 5), reused 16 (delta 2), pack-reused 299
Receiving objects: 100% (322/322), 75.97 MiB | 24.79 MiB/s, done.
Resolving deltas: 100% (150/150), done.


In [1]:
%ls

I40KG-Embeddings/  sample_data/


In [0]:
import scipy
from scipy import spatial
import numpy as np
import math
import json
from scipy.spatial.distance import cdist

### Define function to print the result in tabular format 
The format of tables for Similarity of Frameworks, Standards and Layers are different. Hence, different functions for each type to clear the confusions.

In [0]:
#function to print result in table for similarity of frameworks
def print_result_framework(result):
    print ("{:<8}                                            {:<15}                         {:<10}".format('Framework A','Framework B','Score'))
    print ("----------------------------------------------------------------------------------------------------------------------")
    for key,value in result.items():
        val = str(value)
        val = val.strip("{}")
        val = val.strip("''")
        val = val.replace("':","     ")
        print("{:<8}       {:<15}".format(key +"      ", val))



In [0]:
#function to print result in table for similarity of standards, similarity of standards for same framework and different framework
def print_result_standard(result):
    print ("{:<8}                                            {:<15}                         {:<10}".format('Standard A','Standard B','Score'))
    print ("----------------------------------------------------------------------------------------------------------------------")
    for key,value in result.items():
        val = str(value)
        val = val.strip("{}")
        val = val.strip("''")
        val = val.replace("':","     ")
        print("{:<8}       {:<15}".format(key +"      ", val))



In [0]:
#function to print result in table for similarity for layers
def print_layers(result,framework1,framework2):
    print ("{:<8}                     {:<15}                         {:<10}".format(framework1,framework2,'Score'))
    print ("----------------------------------------------------------------------------------------------------------------------")
    for key,value in result.items():
        val = str(value)
        val = val.strip("{}")
        val = val.strip("''")
        val = val.replace("':","     ")
        print("{:<8}       {:<15}".format(key +"      ", val))


### Define similarity function
We are going to us cosine distance to measure the similarity between two embeddings

In [0]:
#function to calculate cosine distance 
def cosine_similarity(vec1,vec2):
    sum11, sum12, sum22 = 0, 0, 0
    for i in range(len(vec1)):
        x = vec1[i]; y = vec2[i]
        sum11 += x*x
        sum22 += y*y
        sum12 += x*y
    return sum12/math.sqrt(sum11*sum22)

## Similarity among I40 Standarization Frameworks
In this section we show the analysis of similarity among standarization frameworks

In [7]:
!pip3 install rdflib

In [0]:
import json
from rdflib import Graph
import pprint

In [0]:
embeddings_type = "TransR" #Values are TransE, TransD, TransH, TransR
embeddings_path = "/content/I40KG-Embeddings/embeddings/"+embeddings_type+"/sto-enriched"

###Similarity among Frameworks
In this section we analyze the similarity of I4.0 frameworks at the high level, i.e., StandardizationFramework.


In [19]:
g = Graph()
g.parse("/content/I40KG-Embeddings/sto/sto.nt", format="nt")
    
len(g) # prints 2
 #check printing of the graph    
'''for stmt in g:
    pprint.pprint(stmt)'''

#query to get the framework/standard from the sto.nt file
#to get standards we have to replace sto:StandardizationFramework by sto:Standard in the query
#we can get standards of the frameworks as well by just changing the query

qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:StandardizationFramework .
            } limit 1000""")
    
       
#to get the corresponding embeddings of the frameworks/standards from the json file 
with open(embeddings_path+"/entities_to_embeddings.json",'rb') as f:
    array = json.load(f)
new_dict = {}
for row in qres:
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
            #print(new_dict)

#to put the frameworks/standards with their corresponding embeddings in a file            
with open('/content/I40KG-Embeddings/output_framework.json','w') as f:
    # this would place the entire output on one line
    # use json.dump(new_dict, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(new_dict, f)
    
    
#to read the file containing standards/frameworks along with their embeddings   
with open('/content/I40KG-Embeddings/output_framework.json', 'r') as f:
    array = json.load(f)
    
#compare each standard/framework with all the other standards/frameworks to find cosine similarity
result = {}
for key,value in array.items():
    temp,tempDict= 0,{}
    for keyC,valueC in array.items():
        if keyC == key:
            continue
        temp = scipy.spatial.distance.cosine(value,valueC) #send the values of the standards/frameworks to cosine similarity function
        tempDict[keyC] = temp
        val1 = min(tempDict, key=tempDict.get)
    res = {}
    res[val1] = tempDict[val1]
    #print (res)
    result[key]= res
    #result[key]= tempDict
        
#print the standards/frameworks along with their similar standards/frameworks and their similarity distance
print_result_framework(result)

Framework A                                            Framework B                             Score     
----------------------------------------------------------------------------------------------------------------------
https://w3id.org/i40/sto#IIRC             https://w3id.org/i40/sto#IndustrialDataSpace      0.760237478482966
https://w3id.org/i40/sto#AdministrationShell             https://w3id.org/ids/core/TrustedConnector      0.7235714309395047
https://w3id.org/i40/rami#Body             https://w3id.org/i40/sto#NISTInitiative      0.8186195359479508
https://w3id.org/i40/sto#RAMI             https://w3id.org/i40/sto#ISA95      0.7320519245390924
https://w3id.org/i40/sto#ISA95             https://w3id.org/ids/core/TrustedConnector      0.5309462912895939
https://w3id.org/i40/rami#Header             https://w3id.org/i40/sto#IndustrialDataSpace      0.7687442933625579
https://w3id.org/i40/sto#BdvaSira             https://w3id.org/i40/sto#IndustrialDataSpace      0.779394409598773

### Similarity among Standards

In this section we analyze the similarity of I4.0 standards at the high level, i.e., Standard.

In [0]:
g = Graph()
g.parse("/content/I40KG-Embeddings/sto/sto.nt", format="nt")
    
len(g) # prints 2
 #check printing of the graph    
'''for stmt in g:
    pprint.pprint(stmt)'''

#query to get the framework/standard from the sto.nt file
#to get standards we have to replace sto:StandardizationFramework by sto:Standard in the query
#we can get standards of the frameworks as well by just changing the query

qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
            } limit 1000""")
    
       
#to get the corresponding embeddings of the frameworks/standards from the json file 
with open(embeddings_path+"/entities_to_embeddings.json",'rb') as f:
    array = json.load(f)
new_dict = {}
for row in qres:
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
    print(new_dict)

#to put the frameworks/standards with their corresponding embeddings in a file            
with open('/content/I40KG-Embeddings/output_standard.json','w') as f:
    # this would place the entire output on one line
    # use json.dump(new_dict, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(new_dict, f)
    
    
#to read the file containing standards/frameworks along with their embeddings   
with open('/content/I40KG-Embeddings/output_standard.json', 'r') as f:
    array = json.load(f)
    
#compare each standard/framework with all the other standards/frameworks to find cosine similarity
result = {}
for key,value in array.items():
    temp,tempDict= 0,{}
    for keyC,valueC in array.items():
        if keyC == key:
            continue
        temp = scipy.spatial.distance.cosine(value,valueC) #send the values of the standards/frameworks to cosine similarity function
        tempDict[keyC] = temp
        val1 = min(tempDict, key=tempDict.get)
    res = {}
    res[val1] = tempDict[val1]
    #print (res)
    result[key]= res
    #result[key]= tempDict
        
#print the standards/frameworks along with their similar standards/frameworks and their similarity distance
print_result_standard(result)

####Results
The following table shows the frameworks with high degree of similarites.

| Framework A | Framework B | Score |
| ----------------------|:---------------------:| ----------:|
| RAMI 4.0 |  IIRA  | 0.75 |


In [0]:
# TODO implement a fuction to print a table as the one above

###Similarity among different layers of the Standards
In this section we analyze the similarity among layers of different standards, e.g., RAMI 4.0 -> Layer A vs IIRA -> Layer X

In [0]:

g = Graph()
g.parse("/content/I40KG-Embeddings/sto/sto.nt", format="nt")
    
len(g) # prints 2
#check the printing of the graph
'''for stmt in g:
    pprint.pprint(stmt)'''


#queries to get the layers from the sto.nt file for the corresponding Frameworks   
qres1 = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select DISTINCT ?c where {
            ?s sto:hasClassification ?c .
            ?c sto:isDescribedin <https://w3id.org/i40/sto#RAMI> .
            } limit 1000""")

qres2 = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
     select DISTINCT ?c where {
            ?s sto:hasClassification ?c .
            ?c sto:isDescribedin <https://w3id.org/i40/sto#ISA95> .
            } limit 1000""") 


#to get the corresponding embeddings of the layers of two Frameworks from the json file and put it in two different dictionaries
with open(embeddings_path+"/entities_to_embeddings.json",'rb') as f:
    array = json.load(f)
new_dict1 = {}
new_dict2 = {}
for row in qres1:
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict1[tem] = array[key] 
print(new_dict1)
for row in qres2:
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict2[tem] = array[key] 
print(new_dict2)

#the frameworks of which the similarity of their layers will be compared    
framework1 = 'https://w3id.org/i40/sto#RAMI'    
framework2 = 'https://w3id.org/i40/sto#ISA95'

#compare each layers of one Framework  with all the layers of the other Framework to find cosine similarity
result = {}
for key,value in new_dict1.items():
    temp,tempDict= 0,{}
    for keyC,valueC in new_dict2.items():
        if keyC == key:
            continue
        temp = scipy.spatial.distance.cosine(value,valueC)  #pass the vales to find cosine similarity
        tempDict[keyC] = temp
        val1 = min(tempDict, key=tempDict.get)
    res = {}
    res[val1] = tempDict[val1]
    #print (res)
    result[key]= res

#print the similarity of layers of the two Frameworks in tabular format along with their similarity distance
print_layers(result,framework1,framework2)

####Results
The following table shows the similarity among different layers of two frameworks.

| RAMI4.0 | IIRA | Score |
| --------------|:-------:| ----------:|
| Layer A |  Layer X  | 0.85 |
| Layer B |  Layer Y  | 0.75 |
| Layer C |  Layer Z  | 0.85 |


In [0]:
# TODO print a table like the above

##Similarity among Standards of the same Framework
In this section we show the analysis of similarity among standards belonging to the same framework.

In [0]:

g = Graph()
g.parse("/content/I40KG-Embeddings/sto/sto.nt", format="nt")
    
len(g) # prints 2
#check for printing of the graph    
'''for stmt in g:
    pprint.pprint(stmt)'''

#query to get the standards of a Framework from the sto.nt file
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
            ?s sto:hasClassification ?c .
            ?c sto:isDescribedin <https://w3id.org/i40/sto#RAMI> .
            } limit 1000""")
        
#get the corresponding embeddings of the standards of the given Framework
with open(embeddings_path+"/entities_to_embeddings.json",'rb') as f:
    array = json.load(f)
new_dict = {}
for row in qres:
#for line in oldfile:
    #line1 = line.strip("\n")
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
    print(new_dict)

#to put the standards with their corresponding embeddings in a file
with open('/content/I40KG-Embeddings/output_standard_same_framework.json','w') as f:
    # this would place the entire output on one line
    # use json.dump(lista_items, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(new_dict, f) 

    
#to read the file containing standards along with their embeddings   
with open('/content/I40KG-Embeddings/output_standard_same_framework.json', 'r') as f:
    array = json.load(f)
    
#compare each standard with all the other standards of that Framework to find cosine similarity
result = {}
for key,value in array.items():
    temp,tempDict= 0,{}
    for keyC,valueC in array.items():
        if keyC == key:
            continue
        temp = scipy.spatial.distance.cosine(value,valueC) #send the values of the standards to cosine similarity function
        tempDict[keyC] = temp
        val1 = min(tempDict, key=tempDict.get)
    res = {}
    res[val1] = tempDict[val1]
    #print (res)
    result[key]= res
    #result[key]= tempDict
        
#print the standards along with their similar standards and their similarity distance
print_result_standard(result)

##Similarity among Standards of different Framework
In this section we show the analysis of similarity among standards belonging to different frameworks.

In [0]:

g = Graph()
g.parse("/content/I40KG-Embeddings/sto/sto.nt", format="nt")
    
len(g) # prints 2
#check for printing of the graph    
'''for stmt in g:
    pprint.pprint(stmt)'''

#query to get the standards of two different Frameworks from the sto.nt file    
qres1 = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
            ?s sto:hasClassification ?c .
            ?c sto:isDescribedin <https://w3id.org/i40/sto#RAMI> .
            } limit 10""")

qres2 = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
            ?s sto:hasClassification ?c .
            ?c sto:isDescribedin <https://w3id.org/i40/sto#ISA95> .
            } limit 10""") 

#get the corresponding embeddings of the standards of the given Framework        
with open(embeddings_path+"/entities_to_embeddings.json",'rb') as f:
    array = json.load(f)
new_dict = {}
for row in qres1:
#for line in oldfile:
    #line1 = line.strip("\n")
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
    print(new_dict)
for row in qres2:
#for line in oldfile:
    #line1 = line.strip("\n")
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
    print(new_dict)


#to put the standards with their corresponding embeddings in a file
with open('/content/I40KG-Embeddings/output_standard_different_framework.json','w') as f:
    # this would place the entire output on one line
    # use json.dump(lista_items, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(new_dict, f)
    
    
#to read the file containing standards along with their embeddings   
with open('/content/I40KG-Embeddings/output_standard_different_framework.json', 'r') as f:
    array = json.load(f)
    
#compare each standard with all the other standards to find cosine similarity
result = {}
for key,value in array.items():
    temp,tempDict= 0,{}
    for keyC,valueC in array.items():
        if keyC == key:
            continue
        temp = scipy.spatial.distance.cosine(value,valueC) #send the values of the standards to cosine similarity function
        tempDict[keyC] = temp
        val1 = min(tempDict, key=tempDict.get)
    res = {}
    res[val1] = tempDict[val1]
    #print (res)
    result[key]= res
    #result[key]= tempDict
        
#print the standards of along with their similar standards and their similarity distance
print_result_standard(result)


####Results
The following table shows the similarity among different layers of two frameworks.

| RAMI4.0 | IIRA | Score |
| --------------|:-------:| ----------:|
| Standard A |  Standard X  | 0.85 |
| Standard B |  Standard Y  | 0.75 |
| Standard C |  Standard Z  | 0.85 |
